In [3]:
import numpy as np
import pandas as pd

In [4]:
organs = np.array(["bra", "hea", "int", "liv", "lun", "mus", "ski"])

fpkm_data = pd.read_csv("data/fpkm_all_anno.xls", sep='\t', header=0, low_memory=False)
gene_lists = {}
for organ in organs:
    file_name = "data/deg_id/{}_union_deg_list.csv".format(organ)
    gene_lists[organ] = pd.read_csv(file_name, sep=',', header=0)
fpkm_data = fpkm_data.set_index("name")

In [6]:
import itertools

def extract_df(fpkm_data, x, gene_set, filter=False):
    sample_name_list = []
    for sample_name in fpkm_data.columns:
        if (x in sample_name):
            sample_name_list.append(sample_name)
    if filter:
        # print(fpkm_data[sample_name_list].loc[gene_set])
        # print(np.min(fpkm_data[sample_name_list].loc[gene_set], axis=1) > 1)
        gene_set = gene_set[np.min(fpkm_data[sample_name_list].loc[gene_set], axis=1) > 5]
        # gene_set = gene_set[]

    sample_name_list.append("GeneName")
    return fpkm_data[sample_name_list].loc[gene_set].set_index('GeneName'), gene_set

for x, y in itertools.permutations(organs, 2):
    gene_set = np.union1d(np.array(gene_lists[x]["{}_degs_uni".format(x)]), np.array(gene_lists[y]["{}_degs_uni".format(y)]))
    # build x
    # print(x, y)
    df_x, gene_set = extract_df(fpkm_data, x, gene_set, filter=True) 
    df_y, gene_set = extract_df(fpkm_data, y, gene_set)
    df_x.T.to_csv("data/input/{}_vs_{}-{}.csv".format(x, y, x), index_label=False, sep="\t")
    df_y.T.to_csv("data/input/{}_vs_{}-{}.csv".format(x, y, y), index_label=False, sep="\t")

In [26]:
sample_list = {}
for x in organs:
    now_list = []
    for sample_name in fpkm_data.columns:
        if (x in sample_name):
            now_list.append(sample_name)
    sample_list[x] = now_list
    # fpkm_data[x] = fpkm_data[now_list]
    # print(fpkm_data[now_list].astype("float64", errors='ignore').iloc[5:])
    fpkm_data[x] = fpkm_data[now_list].iloc[5:].astype("float64", errors='ignore').mean(axis=1)
    # print(fpkm_data[x])

In [31]:
for x, y in itertools.permutations(organs, 2):
    candidate_gene = pd.read_csv("result/{} X {} ranked by sig score".format(x, y), sep="\t").set_index("Gene_symbol")
    
    result = pd.merge(candidate_gene, fpkm_data, how="left", left_index=True, right_on="GeneName").reset_index().set_index("GeneName")
    result = result.loc[result[sample_list[x]].astype('float64').mean(axis=1) == result[organs].astype("float64").max(axis=1)]
    col_list = np.concatenate((['Ssec', 'name'], result.columns[170:], sample_list[x], sample_list[y]))
    result[col_list].to_excel("result/filtered_genes/{}_x_{}_filtered_genes.xls".format(x, y))
    


/var/folders/pt/9f05dd_s4_lfhz62v29wfmtc0000gn/T/ipykernel_26389/3692384984.py:10: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  result[col_list].to_excel("result/filtered_genes/{}_x_{}_filtered_genes.xls".format(x, y))
/var/folders/pt/9f05dd_s4_lfhz62v29wfmtc0000gn/T/ipykernel_26389/3692384984.py:10: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xl